In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/clear-subset/evaluate_Corpus_subset.json
/kaggle/input/clear-subset/test_queries_subset
/kaggle/input/clear-subset/str_queries_answers.json
/kaggle/input/clear-subset/evaluate_rel_doc_subset
/kaggle/input/clear-subset/reformulated_Corpus_dict_subset.json
/kaggle/input/clear-subset/Passage_dict_subset
/kaggle/input/clear-subset/evaluate_rel_doc_subset.json
/kaggle/input/clear-subset/df_train_filtered_subset_2023-11-15_20-11-30.json
/kaggle/input/clear-subset/df_train_filtered_subset_reformulated.json
/kaggle/input/clear-subset/test_rel_doc_subset.json
/kaggle/input/clear-subset/str_test_answers_subset.json
/kaggle/input/clear-subset/evaluate_queries_subset
/kaggle/input/clear-subset/df_train_filtered_subset_reformulated_v1.json
/kaggle/input/clear-subset/str_test_corpus_subset.json
/kaggle/input/clear-subset/evaluate_Corpus_subset
/kaggle/input/clear-subset/Passage_dict_subset.json
/kaggle/input/clear-subset/str_queries.json
/kaggle/input/clear-subset/test_Corpus_subset.js

In [2]:
def read_json_with_frozenset_column(path, frozenset_column_name):
  df = pd.read_json(path, orient='records')
  df[frozenset_column_name] = df[frozenset_column_name].apply(lambda x: frozenset(x))
  return df

In [3]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=c12214f3ef5213cfad0a06ec89db2633e3cdba3d0d95b7faba7cf92bf1e2983a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Jan  5 23:44:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              27W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, LoggingHandler, util, models, evaluation, losses, InputExample
import logging
from datetime import datetime
import gzip
import os
import tarfile
from collections import defaultdict
from torch.utils.data import IterableDataset

import json
from sentence_transformers import SentenceTransformer, util
import time
import torch

from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator

import pandas as pd
import random
from random import sample

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout
train_batch_size = 64 #Increasing the train batch size improves the model performance, but requires more GPU memory

# The  model we want to fine-tune


# model_name = 'distilroberta-base'
# word_embedding_model = models.Transformer(model_name, max_seq_length=350)
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
# model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# model_name = 'bert-base-uncased'




# num_dev_queries = 500           #Number of queries we want to use to evaluate the performance while training
# num_max_dev_negatives = 200     #For every dev query, we use up to 200 hard negatives and add them to the dev corpus

# We construct the SentenceTransformer bi-encoder from scratch


# model_name = "/content/drive/MyDrive/SBRT_output/training_biker_bi-encoder-min_5_max_10_ir_10_distilroberta-base_30_iter"
# model = SentenceTransformer(model_name)


cross_encoder = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-6')

model_save_path = '/kaggle/working/CLEAR/model/cross-encoder-30_iter_TinyBERT'+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [7]:
data_folder = '/kaggle/input/clear-subset/'

df_train = read_json_with_frozenset_column('/kaggle/input/clear-subset/df_train_filtered_subset_reformulated_v1.json', 'set_answer')
corpus = {}
title_list = df_train['title'].to_list()

for idx, tmp in enumerate(title_list):
  corpus[str(idx)] = tmp

print('len corpus:', len(corpus))
print(corpus)


Passage_dict = {}
collection_filepath = os.path.join(data_folder, 'Passage_dict_subset.json')
with open(collection_filepath, 'r', encoding='utf8') as fIn:
  Passage_dict = json.load(fIn)

evaluate_queries = {}
collection_filepath = os.path.join(data_folder, 'evaluate_queries_subset.json')
with open(collection_filepath, 'r', encoding='utf8') as fIn:
  evaluate_queries = json.load(fIn)

evaluate_queries_sampled = sample(evaluate_queries, 42)

evaluate_Passage = {}
for idx in evaluate_queries_sampled:
  evaluate_Passage[str(idx)] = Passage_dict[str(idx)]

print(evaluate_Passage)

def get_triplets(Passage_dict):
    Triplets= []
    for k, v in Passage_dict.items():
        for x in v[0]:
            for y in v[1]:
                # query,same_appi,diff_api
                Triplets.append([k,x,y])
    return Triplets

train_triplets = get_triplets(Passage_dict)
evaluate_triplets = get_triplets(evaluate_Passage)

len corpus: 392
{'0': "I am interested in converting milliseconds to nanoseconds in Java. Given the number of milliseconds represents a certain point in time and the conversion is necessary for a specific task, could you provide the relevant Java APIs such as 'java.lang.System.currentTimeMillis' and 'java.lang.System.nanoTime' that I can use to achieve this conversion?", '1': 'Can you provide instructions on utilizing an appropriate Java API to refresh a GUI in SWT after a user clicks a button, specifically tailored to the specific requirements of my application?', '2': 'I find myself frequently concatenating multiple strings in my code (5 concatenations in this scenario, with an average length of X characters each). Is using StringBuilder more advantageous than a concatenation chain, and if so, how would I determine the most efficient solution in such cases?', '3': 'Kindly assist me in retrieving a specific integer value that is stored in an ArrayList. What is the recommended Java API

In [8]:
len(evaluate_queries_sampled)

42

In [9]:
train_dataset=[]
for triplet in train_triplets:
    qid, pos_id, neg_id = triplet

    qid = str(qid)
    pos_id = str(pos_id)
    neg_id = str(neg_id)

    query_text = corpus[qid]
    pos_text = corpus[pos_id]
    neg_text = corpus[neg_id]

    pos_instance = InputExample(texts=[query_text, pos_text],label=1)
    neg_instance = InputExample(texts=[query_text, neg_text],label=0)

    train_dataset.append(pos_instance)
    train_dataset.append(neg_instance)

In [10]:
len(train_dataset)

68340

In [11]:
dev_samples = []
for triplet in evaluate_triplets:
    qid, pos_id, neg_id = triplet

    qid = str(qid)
    pos_id = str(pos_id)
    neg_id = str(neg_id)

    query_text = corpus[qid]
    pos_text = corpus[pos_id]
    neg_text = corpus[neg_id]

    pos_instance = InputExample(texts=[query_text, pos_text],label=1)
    neg_instance = InputExample(texts=[query_text, neg_text],label=0)

    dev_samples.append(pos_instance)
    dev_samples.append(neg_instance)

In [12]:
len(dev_samples)

7000

In [13]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)

evaluator = CEBinaryClassificationEvaluator.from_input_examples(dev_samples, name='cross_encoder')

In [14]:
# Train the model
cross_encoder.fit(train_dataloader=train_dataloader,
          evaluator=evaluator,
          epochs=20,
          evaluation_steps=5000,
          warmup_steps=1000,
          save_best_model=True,
          output_path=model_save_path)

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1068 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1068 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1068 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1068 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/1068 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1068 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1068 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1068 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1068 [00:00<?, ?it/s]